In [ ]:
# For CMP 2023, Drew compiled the land use comparison tables, so this script was
# not used. The numbers were adjusted by a factor to better match MTC's numbers

In [1]:
import pandas as pd
from pathlib import Path

In [2]:
out_dir = r"Q:\MTC\Model\ConsistencyReports\2023\Analysis\Data"
base_year = 2015
forecast_year = 2050

out_dir = Path(out_dir) / "2.Land Use"

In [4]:
mtc_base = pd.read_excel(
    r"Q:\Model Landuse\PBA2050\20210310\PBA50_FinalBlueprintLandUse_TAZdata.xlsx",
    str(base_year),
)
mtc_forecast = pd.read_excel(
    r"Q:\Model Landuse\PBA2050\20210310\PBA50_FinalBlueprintLandUse_TAZdata.xlsx",
    str(forecast_year),
)

In [13]:
sf_base = pd.read_csv(
    r"Y:\champ\landuse\p2021\pba50\2015\2-RunInputsChamp5Parking\tazdata.csv"
)
sf_base = pd.merge(
    sf_base.drop(columns="DISTRICT"),
    mtc_base[["ZONE", "DISTRICT"]].rename(columns={"ZONE": "MTCTAZ"}),
)
sf_forecast = pd.read_csv(
    r"Y:\champ\landuse\p2021\pba50\2050\2-RunInputsChamp5Parking\tazdata.csv"
)
sf_forecast = pd.merge(
    sf_forecast.drop(columns="DISTRICT"),
    mtc_base[["ZONE", "DISTRICT"]].rename(columns={"ZONE": "MTCTAZ"}),
)

In [7]:
sf_base["SD"] = sf_base["DISTRICT"].map(lambda x: x if x > 4 else 1)
sf_forecast["SD"] = sf_forecast["DISTRICT"].map(lambda x: x if x > 4 else 1)

In [12]:
def taz_compare_champ_mtc(sf_taz_df, geog, out_dir, out_filename):
    if geog not in {"COUNTY", "SD"}:
        raise 'geog should be "COUNTY" or "SD".'
    (
        sf_taz_df.groupby(geog)
        .agg(
            {
                "POP": "sum",
                "HHLDS": "sum",
                "GQPOP": "sum",
                "TOTALEMP": "sum",
                "EMPRES": "sum",
            }
        )
        .rename(
            columns={
                "POP": "population",
                "HHLDS": "Households",
                "GQPOP": "Non-Institutional Group Quarters Population",
                "TOTALEMP": "Jobs",
                "EMPRES": "Employed Residents",
            }
        )
    ).to_csv(Path(out_dir) / out_filename)

In [ ]:
taz_compare_champ_mtc(
    sf_base,
    "COUNTY",
    out_dir,
    "Table B.1 Base County Demographics.csv",
)
taz_compare_champ_mtc(
    sf_base,
    "SD",
    out_dir,
    "Table B.2 Base Superdistrict Demographics.csv",
)
taz_compare_champ_mtc(
    sf_forecast,
    "COUNTY",
    out_dir,
    "Table C.1 Forecast County Demographics.csv",
)
taz_compare_champ_mtc(
    sf_forecast,
    "SD",
    out_dir,
    "Table C.2 Forecast Superdistrict Demographics.csv",
)